In [ ]:
!pip install --upgrade --no-cache-dir xlrd
!pip install --upgrade --no-cache-dir gdown
!pip install --upgrade --no-cache-dir scipy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import gdown
import statsmodels.formula.api as smf
import plotly.graph_objects as go

from scipy import stats
from plotly.subplots import make_subplots


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
url = "https://drive.google.com/u/0/uc?id=17dh2vGpFRV_8Sp4dOrFfnHqr0QXzzu0w&export=download"  
output = "/content/sample_data/data.xlsx"
gdown.download(url, output, quiet=True)


'/content/sample_data/grupo8.csv'

In [ ]:
ab_test = pd.read_excel('/content/sample_data/data.xlsx', sheet_name='AB_test', engine='openpyxl')

In [ ]:
next_campaign = pd.read_excel('/content/sample_data/data.xlsx', sheet_name='Next_Campaign', engine='openpyxl')

In [ ]:
next_campaign.head()

,id,channel_acq,num_past_purch,spent_last_purchase,weeks_since_visit,browsing_minutes,shopping_cart
0,1,1,0,0.00,0,16,1
1,2,3,0,0.00,2,19,0
2,3,4,5,63.99,1,14,0
3,4,1,0,0.00,7,15,0
4,5,2,2,52.49,2,19,0


| Variable Name | Decription | Notes |
| :- | :- | :-: |
| id | Unique customer identifier | 
| trans_after | Number of transactions after the experiment | Only available for AB_Test
| revenue_after | Total revenue after the experiment | Only available for AB_Test (USD)
| test_coupon | Whether the user received a	coupon | Only available for AB_Test
| channel_acq | Channel of acquisition for the customer when they first signed up to Artea | 1: Google 2: Facebook 3: Instagram 4: Referral 5: Other 
| num_past_purch | Number of previous purchases  | 
| spent_last_purchase | Total spent in previous purchase  | (USD)
| weeks_since_visit | # weeks since last visit | 
| browsing_minutes | Time spend on website in last visit  | (minutes)
| shopping_cart | Whether the user added a product to the shopping cart in last visit | 


In [ ]:
ab_test.describe()

,id,trans_after,revenue_after,test_coupon,channel_acq,num_past_purch,spent_last_purchase,weeks_since_visit,browsing_minutes,shopping_cart
count,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000
mean,8500.500000,0.138800,7.659324,0.50040,2.068200,2.055600,57.423238,3.220000,13.68900,0.292800
std,1443.520003,0.419487,23.627448,0.50005,1.054595,2.560355,55.466516,2.258632,6.96471,0.455093
min,6001.000000,0.000000,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,1.00000,0.000000
25%,7250.750000,0.000000,0.000000,0.00000,1.000000,0.000000,0.000000,1.000000,9.00000,0.000000
50%,8500.500000,0.000000,0.000000,1.00000,2.000000,1.000000,56.990000,3.000000,14.00000,0.000000
75%,9750.250000,0.000000,0.000000,1.00000,3.000000,3.000000,73.990000,5.000000,19.00000,1.000000
max,11000.000000,5.000000,247.960000,1.00000,5.000000,22.000000,361.960000,7.000000,37.00000,1.000000


In [ ]:
fig = make_subplots(
    rows=2, cols=5,
    row_heights=[0.2, 0.2],
    column_widths=[0.6,0.6,0.6,0.6,0.6],
    shared_yaxes = True,
    row_titles= ["test_coupon = 1","test_coupon = 0"],
    column_titles= ["Google","Facebook","Instagram","Referral","Other"],
    specs=[[{'type':'box'}, {'type':'box'}, {'type':'box'}, {'type':'box'}, {'type':'box'}],
           [{'type':'box'}, {'type':'box'}, {'type':'box'}, {'type':'box'}, {'type':'box'}]
           ])

##Google
fig.add_trace(
    go.Box(y = ab_test.loc[ab_test['channel_acq'] == 1].loc[ab_test['test_coupon'] == 1].iloc[:,2].to_numpy(),name='With Cupom'),
    row=1, col=1
    )

fig.add_trace(
    go.Box(y = ab_test.loc[ab_test['channel_acq'] == 1].loc[ab_test['test_coupon'] == 0].iloc[:,2].to_numpy(),name='Without Cupom'),
    row=2, col=1
    )


##Facebook
fig.add_trace(
    go.Box(y = ab_test.loc[ab_test['channel_acq'] == 2].loc[ab_test['test_coupon'] == 1].iloc[:,2].to_numpy(),name='With Cupom'),
    row=1, col=2
    )

fig.add_trace(
    go.Box(y = ab_test.loc[ab_test['channel_acq'] == 2].loc[ab_test['test_coupon'] == 0].iloc[:,2].to_numpy(),name='Without Cupom'),
    row=2, col=2
    )


##Instagram
fig.add_trace(
    go.Box(y = ab_test.loc[ab_test['channel_acq'] == 3].loc[ab_test['test_coupon'] == 1].iloc[:,2].to_numpy(),name='With Cupom'),
    row=1, col=3
    )

fig.add_trace(
    go.Box(y = ab_test.loc[ab_test['channel_acq'] == 3].loc[ab_test['test_coupon'] == 0].iloc[:,2].to_numpy(),name='Without Cupom'),
    row=2, col=3
    )


##Referral
fig.add_trace(
    go.Box(y = ab_test.loc[ab_test['channel_acq'] == 4].loc[ab_test['test_coupon'] == 1].iloc[:,2].to_numpy(),name='With Cupom'),
    row=1, col=4
    )

fig.add_trace(
    go.Box(y = ab_test.loc[ab_test['channel_acq'] == 4].loc[ab_test['test_coupon'] == 0].iloc[:,2].to_numpy(),name='Without Cupom'),
    row=2, col=4
    )


##Other
fig.add_trace(
    go.Box(y = ab_test.loc[ab_test['channel_acq'] == 5].loc[ab_test['test_coupon'] == 1].iloc[:,2].to_numpy(),name='With Cupom'),
    row=1, col=5
    )

fig.add_trace(
    go.Box(y = ab_test.loc[ab_test['channel_acq'] == 5].loc[ab_test['test_coupon'] == 0].iloc[:,2].to_numpy(),name='Without Cupom'),
    row=2, col=5
    )


fig.update_layout(
    template="plotly_dark",
    margin=dict(r=10, t=25, b=40, l=60),
    showlegend= False
    )

fig.show()

In [ ]:
ab_test.groupby(['channel_acq', 'test_coupon']).mean()

id  trans_after  revenue_after  \
channel_acq test_coupon                                            
1           0            8506.486855     0.078870       4.922288   
            1            8489.151786     0.058532       2.803897   
2           0            8574.833010     0.159223       9.814913   
            1            8399.952206     0.220588      10.895294   
3           0            8548.300637     0.146497       9.171783   
            1            8441.054759     0.204694      10.337346   
4           0            8732.141593     0.185841      11.179558   
            1            8394.312000     0.240000      11.518080   
5           0            8638.637931     0.258621      14.859483   
            1            8653.327586     0.241379      12.756690   

                         num_past_purch  spent_last_purchase  \
channel_acq test_coupon                                        
1           0                  2.017527            57.661392   
            1                  2.004960            57.791181   
2           0                  1.974757            54.100835   
            1                  2.222426            58.036471   
3           0                  2.014013            58.047911   
            1                  2.052151            58.364133   
4           0                  2.256637            51.814425   
            1                  2.304000            57.254480   
5           0                  2.068966            53.577586   
            1                  2.431034            64.800345   

                         weeks_since_visit  browsing_minutes  shopping_cart  
channel_acq test_coupon                                                      
1           0                     3.194742         13.331061       0.321324  
            1                     3.340278         13.287698       0.290675  
2           0                     3.213592         13.234951       0.293204  
            1                     3.058824         12.534926       0.332721  
3           0                     3.154140         14.778344       0.264968  
            1                     3.299870         15.306389       0.242503  
4           0                     3.212389         11.637168       0.345133  
            1                     3.256000         12.176000       0.296000  
5           0                     3.034483         14.103448       0.362069  
            1                     3.103448         12.568966       0.310345

In [ ]:
#after campaign
ab_test.groupby(['channel_acq', 'test_coupon']).sum()

id  trans_after  revenue_after  num_past_purch  \
channel_acq test_coupon                                                        
1           0            8736162           81       5055.190            2072   
            1            8557065           59       2826.328            2021   
2           0            4416039           82       5054.680            1017   
            1            4569574          120       5927.040            1209   
3           0            6710416          115       7199.850            1581   
            1            6474289          157       7928.744            1574   
4           0             986732           21       1263.290             255   
            1            1049289           30       1439.760             288   
5           0             501041           15        861.850             120   
            1             501893           14        739.888             141   

                         spent_last_purchase  weeks_since_visit  \
channel_acq test_coupon                                           
1           0                       59218.25               3281   
            1                       58253.51               3367   
2           0                       27861.93               1655   
            1                       31571.84               1664   
3           0                       45567.61               2476   
            1                       44765.29               2531   
4           0                        5855.03                363   
            1                        7156.81                407   
5           0                        3107.50                176   
            1                        3758.42                180   

                         browsing_minutes  shopping_cart  
channel_acq test_coupon                                   
1           0                       13691            330  
            1                       13394            293  
2           0                        6816            151  
            1                        6819            181  
3           0                       11601            208  
            1                       11740            186  
4           0                        1315             39  
            1                        1522             37  
5           0                         818             21  
            1                         729             18



*   H0 =image.pngrevenue_after = image.pngrevenue_after sem coupom

*   H1 = image.pngrevenue_after > image.pngrevenue_after sem coupom


In [ ]:
ab_test.loc[ab_test['test_coupon'] == 1].iloc[:,:]

,id,trans_after,revenue_after,test_coupon,channel_acq,num_past_purch,spent_last_purchase,weeks_since_visit,browsing_minutes,shopping_cart
1,6002,0,0.0,1,1,2,53.99,0,7,1
2,6003,0,0.0,1,2,3,88.98,3,4,0
4,6005,0,0.0,1,3,3,66.49,4,20,0
6,6007,0,0.0,1,3,1,34.99,5,20,0
9,6010,0,0.0,1,3,5,57.49,3,17,0
...,...,...,...,...,...,...,...,...,...,...
4990,10991,0,0.0,1,3,0,0.00,4,18,0
4993,10994,0,0.0,1,3,4,38.49,7,19,0
4996,10997,0,0.0,1,1,0,0.00,3,8,0
4997,10998,0,0.0,1,1,1,60.49,4,17,0


In [ ]:
treat_pvalue = lambda x: round(100*x, 1)

canal = ab_test#.loc[ab_test['channel_acq'] == 3]

tratamento = canal.loc[ab_test['test_coupon'] == 1].iloc[:,1].to_numpy()
controle   = canal.loc[ab_test['test_coupon'] == 0].iloc[:,1].to_numpy()

t2, pvalor_ttest = stats.ttest_ind(tratamento, controle, equal_var = True, alternative = 'greater')
pvalor_ttest = treat_pvalue(pvalor_ttest)
pvalor_ttest 

1.4